In [128]:
import pandas as pd

from main import ParseReport, fuzz

report = ParseReport(file="result/TEST - Magellan - Cloud Example and Explanation.json", json_file=True, save_json=False)
report.get_lien()

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
index 0 is out of bounds for axis 0 with size 0


[{'Lien Type:': 'Federal Tax Lien',
  'Filed Against:': None,
  'Amount:': '$87,052.04',
  'Recorded Date:': '01/10/2020',
  'Recording Information:': None,
  'Comment:': None},
 {'Lien Type:': 'Federal Tax Lien',
  'Filed Against:': None,
  'Amount:': '$35,986.01',
  'Recorded Date:': '03/08/2010',
  'Recording Information:': None,
  'Comment:': None}]

In [129]:
report.words.head()

,page_idx,dimensions,orientation.value,orientation.confidence,language.value,language.confidence,block_idx,block_geometry,artefacts,line_idx,line_geometry,word_idx,value,confidence,x1,y1,x2,y2
index,,,,,,,,,,,,,,,,,,
0,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",0,Southern,0.999931,0.436523,0.056641,0.545898,0.074219
1,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",1,Indiana,0.962854,0.552734,0.056641,0.643555,0.074219
2,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",2,Abstract,0.979812,0.649414,0.058594,0.750977,0.073242
3,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",3,"Company,",0.980987,0.756836,0.057617,0.879883,0.078125
4,0,"[1684, 1190]",None,None,None,None,0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",[],0,"[[0.4365234375, 0.0556640625], [0.9267578125, ...",4,Inc,0.987708,0.886719,0.055664,0.926758,0.075195


In [130]:
context = report.words
word1, word2 = "Notice", "Lien"
line = context[context['value'].str.contains(word1) | context['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]
        
line['paired']=  line.value + " " + line.value.shift(-1)
line['space']=  line.x1.shift(-1) - line.x2
line['align']=  (abs(line.y1.shift(-1) - line.y1) + (line.y2.shift(-1) - line.y2))/2

line["score"] = line["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
line = line[line.score < 10]
line = line.sort_values(by=['score', 'space', "align"])
# line = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
# line = context[(context['page_idx'] == line.page_idx.item()) & (context['block_idx'] == line.block_idx.item()) & (context['line_idx'] == line.line_idx.item())]["value"].values
line

,page_idx,block_idx,line_idx,x1,y1,x2,y2,value,paired,space,align,score
index,,,,,,,,,,,,
2374,13,222,517,0.424805,0.083984,0.485352,0.098633,Notice,Notice Lien,0.133789,0.000000,0
2021,12,187,427,0.420898,0.089844,0.479492,0.101562,Notice,Notice Lien,0.134766,0.001465,0
3020,15,269,681,0.250977,0.227539,0.312500,0.238281,Notices,"Notices Lien,",0.191406,0.092285,8


In [131]:
context = report.words
word1, word2 = "Notice", "Lien"
line = context[context['value'].str.contains(word1) | context['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]

line['paired']=  line.value + " " + line.value.shift(-1)
line['space']=  line.x1.shift(-1) - line.x2
line['align']=  (abs(line.y1.shift(-1) - line.y1) + (line.y2.shift(-1) - line.y2))/2

line["score"] = line["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
line = line[line.score < 10]
line = line.sort_values(by=['score', 'space', "align"])
result = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
result = context[(context['page_idx'] == result.page_idx.item()) & (context['block_idx'] == result.block_idx.item()) & (context['line_idx'] == result.line_idx.item())]["value"].values
result


array(['Notice', 'of', 'Federal', 'Tax', 'Lien'], dtype=object)

In [132]:
result = line.iloc[1][["page_idx", "block_idx", "line_idx", "paired"]]
result = context[(context['page_idx'] == result.page_idx.item()) & (context['block_idx'] == result.block_idx.item()) & (context['line_idx'] == result.line_idx.item())]["value"].values
result

array(['Notice', 'of', 'Federal', 'Tax', 'Lien'], dtype=object)

In [133]:
word1, word2 = "Tax", "Period"
row = line.iloc[0][["page_idx", "block_idx", "line_idx", "paired"]]
c = report.words[(report.words.page_idx == row.page_idx)]
column = c[c['value'].str.contains(word1) | c['value'].str.contains(word2)][["page_idx", "block_idx", "line_idx", "x1", "y1", "x2", "y2", "value"]]
column['paired']=  column.value + " " + column.value.shift(-1)
column['space']=  abs(column.x1.shift(-1) - column.x2)
column['align']=  (abs(column.y1.shift(-1) - column.y1) + (column.y2.shift(-1) - column.y2))/2

column['x12']=  column.x1.shift(-1)
column['y12']=  column.y1.shift(-1)
column['x22']=  column.x2.shift(-1)
column['y22']=  column.y2.shift(-1)

column["score"] = column["paired"].apply(lambda x: 100 - fuzz.ratio(str(x), str(word1) + " " + str(word2)))
column = column[column.score < 10]
column = column.sort_values(by=['score', 'space', "align"])
column

,page_idx,block_idx,line_idx,x1,y1,x2,y2,value,paired,space,align,x12,y12,x22,y22,score
index,,,,,,,,,,,,,,,,
2540,13,232,542,0.270508,0.335938,0.298828,0.347656,Tax,Tax Period,0.000977,0.0,0.299805,0.335938,0.342773,0.347656,0


In [134]:
c[(c.block_idx==231)][["value", "line_idx"]]

,value,line_idx
index,,
2533,in,539
2534,IRC,539
2535,6325(a).,539
2536,Kind,540
2537,of,540
2538,Tax,540
2539,(a),541


In [144]:
right = 0.02
left = 0.02
column_data = c[(c.x1 >= column.x1.item() - left) & (c.x2 <= column.x22.item() + right) & (c.y1 >= column.y2.item())]
column_data['hd']=  abs(column_data.y1.shift(-1) - column.y2.item())
column_data = column_data.sort_values(by=['hd'])
column_data['bid']=  abs(column_data.block_idx.shift(-1) - column_data.block_idx)
column_data['lid']=  abs(column_data.line_idx.shift(-1) - column_data.line_idx)
column_data['h2d']=  abs(column_data.hd.shift(-1) - column_data.hd)
column_data = column_data.sort_values(by=['hd', 'h2d', 'bid', "lid"])
column_data = column_data.reset_index(drop=True)
values = []
for i, k in column_data.iterrows():
    print(k.value, k.bid, k.lid, k.h2d)
    print(k.bid - column_data.iloc[i + 1].bid.item() > 0)
    print(k.lid - column_data.iloc[i + 1].lid.item() > 0)
    print(column_data.iloc[i + 1].h2d.item() > 0.12)
    print(i)
    if i < 2 and (k.bid > 1 or k.lid > 1):
        continue
    elif k.bid > 1 or k.lid > 1 or column_data.iloc[i + 1].h2d.item() > 0.12:
    # print(column_data.bid - column_data.iloc[i + 1].bid.item())
    # print(k.bid - column_data.iloc[i + 1].bid.item())
        break
    else:
        values.append(k.value)

values
column_data

(b) 5.0 16.0 0.0107421875
True
True
False
0
12/31/2010 0.0 1.0 0.01171875
False
False
False
1
12/31/2011 0.0 1.0 0.0107421875
False
False
False
2
12/31/2012 0.0 1.0 0.0107421875
False
False
False
3
12/31/2013 0.0 1.0 0.0126953125
False
False
False
4
12/31/2014 0.0 1.0 0.009765625
False
False
True
5


/tmp/ipykernel_9486/2056189407.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_data['hd']=  abs(column_data.y1.shift(-1) - column.y2.item())


,page_idx,dimensions,orientation.value,orientation.confidence,language.value,language.confidence,block_idx,block_geometry,artefacts,line_idx,...,value,confidence,x1,y1,x2,y2,hd,bid,lid,h2d
0,13,"[1684, 1190]",None,None,None,None,232,"[[0.2705078125, 0.3359375], [0.3427734375, 0.3...",[],543,...,(b),0.976120,0.295898,0.356445,0.321289,0.368164,0.020508,5.0,16.0,0.010742
1,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],559,...,12/31/2010,0.995618,0.258789,0.368164,0.356445,0.381836,0.031250,0.0,1.0,0.011719
2,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],560,...,12/31/2011,0.994220,0.258789,0.378906,0.356445,0.392578,0.042969,0.0,1.0,0.010742
3,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],561,...,12/31/2012,0.849877,0.258789,0.390625,0.357422,0.404297,0.053711,0.0,1.0,0.010742
4,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],562,...,12/31/2013,0.999272,0.258789,0.401367,0.355469,0.415039,0.064453,0.0,1.0,0.012695
5,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],563,...,12/31/2014,0.999373,0.258789,0.412109,0.355469,0.425781,0.077148,0.0,1.0,0.009766
6,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],564,...,12/31/2015,0.991480,0.259766,0.424805,0.355469,0.435547,0.086914,0.0,1.0,0.138672
7,13,"[1684, 1190]",None,None,None,None,237,"[[0.193359375, 0.3681640625], [0.701171875, 0....",[],565,...,12/31/2016,0.995368,0.258789,0.434570,0.356445,0.448242,0.225586,2.0,3.0,0.009766
8,13,"[1684, 1190]",None,None,None,None,239,"[[0.3056640625, 0.5625], [0.49609375, 0.595703...",[],568,...,FLOYD,0.999671,0.306641,0.573242,0.357422,0.583984,0.235352,3.0,5.0,0.038086
9,13,"[1684, 1190]",None,None,None,None,242,"[[0.16796875, 0.6201171875], [0.412109375, 0.6...",[],573,...,prepared,0.999099,0.268555,0.621094,0.324219,0.632812,0.273438,3.0,4.0,0.000000
